## Fine-tuning the GPT Model using Lora fine-tuning technique

1. You can change model_name to a smaller one like EleutherAI/pythia-70m if needed or also you can replace model with ollama, llama 2, etc.,

2. Since we're running it from colab, we should use less weight models for practice

3. This example uses qLoRA (load_in_4bit=True) + LoRA.

In [1]:
!pip install transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.8 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset

# =====================
# Step 1: Dataset
# =====================
data = {
    "text": [
        "Question: What is the capital of France? Answer: Paris.",
        "Question: Who wrote Hamlet? Answer: William Shakespeare.",
        "Question: What is 2 + 2? Answer: 4.",
    ]
}
dataset = Dataset.from_dict(data)

# =====================
# Step 2: Load Model + Tokenizer with qLoRA
# =====================
model_name = "tiiuae/falcon-rw-1b"   # TinyLlama/TinyLlama-1.1B-Chat-v1.0
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

'''
Below QLoRA function prepares a model that was loaded in 4-bit or 8-bit (quantized).

Most weights are in int4 (compressed to save GPU memory).

But some parts (like layer norms, output heads) still need to be in float32 or float16 for training stability.
'''
model = prepare_model_for_kbit_training(model)

# =====================
# Step 3: Apply LoRA
# =====================
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,   # a scaling factor that controls how much the LoRA update influences the original weight. High alpha have a strong influence where as lower alpha will have lesser influence.
    target_modules=["query_key_value"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# =====================
# Step 4: Tokenize Dataset
# =====================
def tokenize(sample):
    return tokenizer(sample["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize)

# =====================
# Step 5: Training
# =====================
training_args = TrainingArguments(
    output_dir="./dummy",  # won't be saved
    per_device_train_batch_size=4, # each GPU trains with 4 examples at once
    num_train_epochs=3,
    learning_rate=2e-4,  # 0.0002
    fp16=True,          # Use half-precision floating point (16 bits instead of 32). Saves GPU memory and speeds things up
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

A data collator takes tokenized examples from your dataset and turns them into mini-batches that the model can consume. it handles,

1. Padding sequences in the batch so they’re the same length.

2. Shifting tokens to create inputs and labels for language modeling.

3. Optionally applying masking(MLM)

mlm=True → Masked Language Modeling (MLM), like BERT training.

Randomly mask tokens (e.g., turn “Paris” → “[MASK]”) and ask model to predict them using the available sentence. (BERT MLM - Guessing the masking word)

mlm=False → Causal Language Modeling (CLM), like GPT/Falcon training. (Here we're predicting the next word)

Model predicts the next token in sequence, without random masking.

Since Falcon is a causal LM, we set mlm=False.

In [3]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #mlm = masked languague model

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipython-input-4195125670.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=3, training_loss=2.051107088724772, metrics={'train_runtime': 53.8862, 'train_samples_per_second': 0.167, 'train_steps_per_second': 0.056, 'total_flos': 8364732973056.0, 'train_loss': 2.051107088724772, 'epoch': 3.0})

In [4]:
# =====================
# Step 6: Generate Output
# =====================
model.eval()
# prompt = "What is 18 + 2? Answer:"
prompt = "What is the capital of France? Answer:"
# prompt = "Who wrote Hamlet? Answer:"
# prompt = "What is black hole?"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=True,   # Enables sampling instead of greedy decoding. Greedy - Always picks the most likely word | Sample - Choose randomness, so answers do vary.
        top_k=50,   # Limits sampling to the 50 most likely next tokens at each step & picks the random one from it.
        top_p=0.9, # Instead of a fixed k, take the smallest set of tokens whose probabilities sum to 90%. If top 3 tokens already cover 90%, only those 3 are considered vice versa for 20tokens covers 90%, all 20 will be considered.
        temperature=0.7 # Lower (<1) = sharper distribution → more deterministic. | Higher (>1) = flatter distribution → more randomness.
    )


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [5]:
print("\n=== Model Output ===")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


=== Model Output ===
What is the capital of France? Answer: Paris.
So, what does this have to do with your SEO? Everything.
The SEO industry has changed a lot in the last 10 years


Push the LoRA Adapters to HuggingFace Hub

In [7]:
!huggingface-cli login

model.push_to_hub("jaich/falcon_1b_LoRA_Adapter_4_bit_GenEnglish", check_pr=True)

tokenizer.push_to_hub("jaich/falcon_1b_LoRA_Adapter_4_bit_GenEnglish",check_pr=True)


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `HF_WRITE_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-cred

adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jaich/falcon_1b_LoRA_Adapter_4_bit_GenEnglish/commit/a784b3803dce33c8f6c811b86d54595929d887ca', commit_message='Upload tokenizer', commit_description='', oid='a784b3803dce33c8f6c811b86d54595929d887ca', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jaich/falcon_1b_LoRA_Adapter_4_bit_GenEnglish', endpoint='https://huggingface.co', repo_type='model', repo_id='jaich/falcon_1b_LoRA_Adapter_4_bit_GenEnglish'), pr_revision=None, pr_num=None)

# What happens when someone loads it back?

1. Load the original base model (7B params).

2. Load your LoRA adapter weights (MBs). (adapter_config.json) ()

3. Combine them at runtime.

So users still need access to the base model checkpoint.

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-rw-1b")
model = PeftModel.from_pretrained(base_model, "jaich/falcon_1b_LoRA_Adapter_4_bit_GenEnglish")

Merge the base model with LoRA updated weight during the training itself

In [ ]:
new_model = "falcon-1B-merged-weight-finetune-LoRA-adapter-weights"

trainer.model.save_pretrained(new_model)

from peft import LoraConfig, PeftModel
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-rw-1b",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-rw-1b", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#Evaluating the Model using BLEU & ROUGE Score

In [ ]:
# =====================
# Step 6: Evaluation with BLEU & ROUGE
# =====================
model.eval()

# Define test prompts and references
eval_data = [
    {"prompt": "What is the capital of France? Answer:", "reference": "Paris."},
    {"prompt": "Who wrote Hamlet? Answer:", "reference": "William Shakespeare."},
    {"prompt": "What is 2 + 2? Answer:", "reference": "4."},
]

predictions = []
references = []

for item in eval_data:
    inputs = tokenizer(item["prompt"], return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature=0.7
        )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the answer part (after "Answer:")
    if "Answer:" in decoded:
        decoded = decoded.split("Answer:")[-1].strip()
    predictions.append(decoded)
    references.append(item["reference"])

print("\n=== Predictions vs References ===")
for p, r in zip(predictions, references):
    print(f"Pred: {p} | Ref: {r}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== Predictions vs References ===
Pred: Paris. And if you’re looking for | Ref: Paris.
Pred: William Shakespeare.
This is one of the most | Ref: William Shakespeare.
Pred: 4.
That’s right, there | Ref: 4.


In [ ]:
predictions

['Paris. And if you’re looking for',
 'William Shakespeare.\nThis is one of the most',
 '4.\nThat’s right, there']

In [ ]:
references

['Paris.', 'William Shakespeare.', '4.']

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4ec8846660f98a454ee1d223e65df73b50b8bb2c8d6a4629591a94f41a9e9e41
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
# =====================
# Compute BLEU & ROUGE
# =====================
import evaluate
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

bleu_score = bleu.compute(predictions=predictions, references=[[r] for r in references])
rouge_score = rouge.compute(predictions=predictions, references=references)

print("\n=== Evaluation Scores ===")
print("BLEU:", bleu_score)
print("ROUGE:", rouge_score)


=== Evaluation Scores ===
BLEU: {'bleu': 0.0, 'precisions': [0.3181818181818182, 0.21052631578947367, 0.0625, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 3.142857142857143, 'translation_length': 22, 'reference_length': 7}
ROUGE: {'rouge1': np.float64(0.3277777777777778), 'rouge2': np.float64(0.08333333333333333), 'rougeL': np.float64(0.3277777777777778), 'rougeLsum': np.float64(0.3277777777777778)}


#Understand the parameters changes using LoRA adapters

In [ ]:
# =====================
# Check Trainable Parameters
# =====================
def print_trainable_parameters(model):
    trainable = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable += param.numel()
    print(f"Trainable params: {trainable:,}")
    print(f"All params: {all_params:,}")
    print(f"Trainable%: {100 * trainable / all_params:.4f}%")

print_trainable_parameters(model)


Trainable params: 1,572,864
All params: 709,218,304
Trainable%: 0.2218%


**Understand the results:**

All params: 709,218,304 → total Falcon weights (frozen + trainable).

Trainable params: 1,572,864 → LoRA adapter weights only.

Trainable %: 0.2218% → you’re updating ~1 in 451 weights (about 0.22%) instead of full fine-tuning.


“We fine-tune just 1.57M / 709M ≈ 0.22% of the model — roughly 1 out of every 451 weights